In [165]:
import pandas as pd
import numpy as np
import pickle
import datetime
import json
import os

import pytz
from dateutil import parser

### Variables needed from user

In [166]:
## Input Directory
directory = "../PaymentTestData"

## Date of first Monday
first_monday = datetime.datetime(2022, 8, 15)

## Days when Export Errors Occured
export_error_days = [datetime.datetime(2022, 8, 17), datetime.datetime(2022, 8, 20)]

### Some other settings

In [167]:
## Display settings
pd.get_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [168]:
## Date of the Sunday of the week
coming_sunday = first_monday + datetime.timedelta(days=6)

### Some Handy Functions
- To convert UTC to ET
- To pivot table
- To insert missing columns
- To make the payment DataFrame

In [169]:
## Function to fix date from UTC to ET
def fix_date_to_ET(dataframe, date_col_name):
    # Add date column to samples dataframe
    df = dataframe
    df[date_col_name] = 0
    
    for i in range(len(df.DueDate)):
        df.iloc[i]
        if pd.to_datetime(df.DueDate[i], format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').hour < 5:
            # subtract one day from date
            df.loc[i, date_col_name] = pd.to_datetime(df.DueDate[i], format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date() - datetime.timedelta(days=1)
        else:
            df.loc[i, date_col_name] = pd.to_datetime(df.DueDate[i], format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date()
    
    return df

## Functon to pivot table
def pivot_df(dataframe, pos: list, col, val):
    df = dataframe
    df = df.pivot_table(index=pos,
                    columns=col, 
                    values=val).reset_index()
    return df

def fix_columns_by_category(dataframe, categories):
    df = dataframe
    for item in categories:
        if item not in df.columns.to_list():
            df[item] = 'Incomplete'
    return df

## Function to create the custom dataframe
def get_payment_info_dataframe(directory):
    df_survey_list = []
    for folder in os.listdir(directory):
        
        ## Make the survey dataframe
        survey_file_name = ""
        path = directory + "/" + folder
        for f_name in os.listdir(path):
            if f_name.startswith('SurveyTasks'):
                survey_file_name = f_name
                break
        path = path + '/' + survey_file_name
        current_survey_df = pd.read_csv(path) 
        
        ## Make the participant dataframe
        participant_file_name = ""
        path = directory + "/" + folder
        for f_name in os.listdir(path):
            if f_name.startswith('StudyParticipants'):
                participant_file_name = f_name
                break
        path = path + '/' + participant_file_name
        current_participant_df = pd.read_csv(path) 
        
        ## Select the rows where the experiment version is 'app_pilot_1'
        current_participant_df['ExpVersion'] = current_participant_df['CustomFields'].apply(json.loads)
        current_participant_df = current_participant_df.assign(exp_version=lambda x: x.ExpVersion.apply(lambda x: x['exp_version']))
        current_participant_df = current_participant_df[['ParticipantIdentifier', 'UtcOffset', 'TimeZone', 'exp_version']]
        current_participant_df = current_participant_df.loc[current_participant_df.exp_version=='app_pilot_1',]
        
        ## Merge the current participant and SurveyTasks df to get df_temp
        df_temp = pd.merge(current_participant_df, current_survey_df, on='ParticipantIdentifier').reset_index()
        df_survey_list.append(df_temp)
        
    df_survey = pd.concat(df_survey_list).reset_index(drop=True)
    return df_survey

### Make DataFrame from which Payment Info will be calculated

In [170]:
## Got all 'PaymentInformation' till date in a dataframe
df_survey = get_payment_info_dataframe(directory)

In [171]:
# Make SurveyDay column
df = df_survey
df['SurveyDay'] = None
for i in df.index:
    iso = parser.isoparse(df.InsertedDate[i])
    date = iso.astimezone(pytz.timezone(df.TimeZone[i])).date()
    df.SurveyDay[i] = (pd.to_datetime(date)) #.tz_localize('US/Eastern')).date
df_survey = df

/var/folders/57/9mrlx9kx6xsfb5tsnbtdb9d40000gn/T/ipykernel_39025/3959980374.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.SurveyDay[i] = (pd.to_datetime(date)) #.tz_localize('US/Eastern')).date


In [172]:
## Only get rows for the current week
cols = df_survey['SurveyDay']>=pd.to_datetime(first_monday) #.tz_localize('US/Eastern')
df = df_survey[cols]
df_survey = df

cols = df_survey['SurveyDay']<=pd.to_datetime(coming_sunday) #.tz_localize('US/Eastern')
df = df_survey[cols]
df_survey = df

## Drop duplicate columns
df_survey = df_survey.drop_duplicates(subset=['ParticipantIdentifier', 'SurveyDay', 'SurveyName'])

### Get information useful for iterating over DataFrame
- participant_list
- evening_task list
- other_task list
- evening_task + other_task = all_task list

In [173]:
## Get Participant List
participant_list = []
for i in df_survey.index:
    if df_survey.ParticipantIdentifier[i] not in participant_list:
        participant_list.append(df_survey.ParticipantIdentifier[i])

In [174]:
## Evening task names
evening_tasks = ['survey_daily-check-in-detail', 'survey_daily-affect-custom-evening', 'survey_daily-check-in-general']

## Other task names
do_not_include = ['Baseline Demographic Status', 'survey_daily-affect-custom-morning', 'survey_monthly-goal_set', 'survey_monthly-ideal-day',
                  'survey_weekly-goal_set']
other_tasks = []
for index, row in df_survey.iterrows():
    if row['SurveyName'] not in evening_tasks:
        if row['SurveyName'] not in do_not_include:
            if row['SurveyName'] not in other_tasks:
                other_tasks.append(row['SurveyName'])

## All task names
all_tasks = evening_tasks + other_tasks

### Make the payment amount dictionary

In [175]:
## Initialize payment amount doctionary
participant_payment_dict = {}
participant_evening_task_count_dict = {}
participant_morning_task_count_dict = {}
participant_all_task_count_dict = {}
for item in participant_list:
    participant_payment_dict[item] = 0
    participant_evening_task_count_dict[item] = 0
    participant_morning_task_count_dict[item] = 0
    participant_all_task_count_dict[item] = 0

In [176]:
## Get evening payment information
temp_df = []
for item in participant_list:
    temp_df = df_survey.loc[df_survey['ParticipantIdentifier'] == item]
    ## Got one participant df
    temp_df = temp_df[['ParticipantIdentifier', 'SurveyName', 'Status', 'SurveyDay']]

    ## Get df for participant with the evening tasks only
    temp_df = temp_df.loc[temp_df['SurveyName'].isin(evening_tasks)]
    
    ## Pivot table
    temp_df.drop_duplicates()
    temp_df = temp_df.pivot_table(index=['SurveyDay', 'ParticipantIdentifier'],
                        columns='SurveyName', 
                        values='Status',
                        aggfunc=lambda x: ' '.join(x)).reset_index()
    
    ## Account for missing evening task columns
    temp_df = fix_columns_by_category(temp_df, evening_tasks)

    ## Loop over this participant df to calculate the 
    ## number of days all evening tasks completed
    for index, row in temp_df.iterrows():
        if row['survey_daily-affect-custom-evening'] == 'Complete':
            if row['survey_daily-check-in-detail'] == 'Complete':
                if row['survey_daily-check-in-general'] == 'Complete':
                    participant_payment_dict[item] +=1
                    participant_evening_task_count_dict[item] += 1
    
    ## Pay the participant for days not recorded due to export errors
    export_error_payment_count = 0
    curr_participant_date_list = temp_df['SurveyDay'].to_list()
    for day in export_error_days:
        if day not in curr_participant_date_list:
            export_error_payment_count += 1
    
    participant_payment_dict[item] += export_error_payment_count
    participant_evening_task_count_dict[item] += export_error_payment_count

In [177]:
## Get morning payment information
for item in participant_list:
    temp_df = df_survey.loc[df_survey['ParticipantIdentifier'] == item]
    ## Got one participant df
    temp_df = temp_df[['ParticipantIdentifier', 'SurveyName', 'Status', 'SurveyDay']]

    ## Get df for participant with the evening tasks only
    temp_df = temp_df.loc[temp_df['SurveyName'].isin(other_tasks)]
    
    ## Pivot table
    temp_df.drop_duplicates()
    temp_df = temp_df.pivot_table(index=['SurveyDay', 'ParticipantIdentifier'],
                        columns='SurveyName', 
                        values='Status',
                        aggfunc=lambda x: ' '.join(x)).reset_index()
    
    ## Account for missing evening task columns
    temp_df = fix_columns_by_category(temp_df, other_tasks)

    ## Loop over this participant df to calculate the 
    ## number of days all evening tasks completed
    for index, row in temp_df.iterrows():
        count = 0 ## total number of non evening task is 13
        for task in other_tasks:
            if row[task] == 'Complete':
                count +=1
        if count >= 13:
            participant_payment_dict[item] +=1
            participant_morning_task_count_dict[item] += 1
    
    ## Pay the participant for days not recorded due to export errors
    export_error_payment_count = 0
    curr_participant_date_list = temp_df['SurveyDay'].to_list()
    for day in export_error_days:
        if day not in curr_participant_date_list:
            export_error_payment_count += 1
    
    participant_payment_dict[item] += export_error_payment_count
    participant_morning_task_count_dict[item] += export_error_payment_count

### Lottery Entry Calculator
- For each participant, iterate over the days
- If at any day the participant did not do all tasks, then deduct a lottery entry

In [178]:
## Initialize Lottery amount doctionary
lottery_entries = {}
for item in participant_list:
    lottery_entries[item] = 3
    
## Get evening payment information
for item in participant_list:
    temp_df = df_survey.loc[df_survey['ParticipantIdentifier'] == item]
    
    ## Got one participant df
    temp_df = temp_df[['ParticipantIdentifier', 'SurveyName', 'Status', 'SurveyDay']]

    ## Get df for participant with the evening tasks only
    temp_df = temp_df.loc[temp_df['SurveyName'].isin(all_tasks)]
    
    ## Pivot table
    temp_df.drop_duplicates()
    temp_df = temp_df.pivot_table(index=['SurveyDay', 'ParticipantIdentifier'],
                        columns='SurveyName', 
                        values='Status',
                        aggfunc=lambda x: ' '.join(x)).reset_index()
    
    ## Account for missing evening task columns
    temp_df = fix_columns_by_category(temp_df, all_tasks)

    ## Loop over this participant df to calculate the 
    ## number of days all evening tasks completed
    for index, row in temp_df.iterrows():
        count = 0 ## total number of non evening task is 13
        for task in all_tasks:
            if row[task] == 'Complete':
                count +=1
        if count < 16:
            lottery_entries[row['ParticipantIdentifier']] -=1
    
    ## Account for completely missed days
    deductor = (len(temp_df)) + (len(export_error_days))
    if deductor > 7:
        deductor = 0
    else:
        deductor = 7 - deductor
    lottery_entries[row['ParticipantIdentifier']] -=deductor

# If they dont complete three or more days, they should have 0 entries
for item in lottery_entries.keys():
    if lottery_entries[item] < 0:
        lottery_entries[item] = 0

### Final DataFrame with Lottery entries, Payment amount, and Identifier

In [179]:
## Make final dataframe
payment_dataframe = []
for item in participant_list:
    temp = {}
    temp['ParticipantIdentifier'] = item
    temp['LotteryEntries'] = lottery_entries[item]
    temp['No_Evenings_Completed'] = participant_evening_task_count_dict[item]
    temp['No_Mornings_Completed'] = participant_morning_task_count_dict[item]
    ## temp['No_Days_Completed'] = participant_all_task_count_dict[item]
    temp['Payment Amount ($)'] = participant_payment_dict[item] * 1.5
    payment_dataframe.append(temp)
payment_dataframe = pd.DataFrame(payment_dataframe)

In [180]:
payment_dataframe

,ParticipantIdentifier,LotteryEntries,No_Evenings_Completed,No_Mornings_Completed,Payment Amount ($)
0,48396d3f-058f-4526-b7e7-b260c92774ea,1,6,5,16.5
1,d9139666-0f9d-42b3-9ec4-c8d9d0a69aa8,0,5,5,15.0
2,c8892869-a3c8-4dfa-be25-3220d10f48c6,3,7,7,21.0
3,2b10f0e0-ef05-41a6-99cb-a543d0e6cb4e,3,7,7,21.0
4,a2a5c70f-4d45-4f4a-bec5-5e2c1776689b,1,6,6,18.0
5,79939a12-9002-468d-a41b-15c3b66d3de5,2,6,6,18.0
6,f1ee447d-70b3-4228-8f9b-2f6a8f2391fb,0,0,0,0.0
7,a4e70990-a323-4974-acbc-a90d230b74fa,0,4,5,13.5
8,64ea1889-52d2-409b-a954-fed089584380,0,3,4,10.5
9,8a5145e5-6977-4beb-b7be-18e4ce6451bb,1,5,5,15.0


In [139]:
## Individial participant completion sanity check
# cols = df_survey['ParticipantIdentifier'] == 'd88ca7d8-2aa4-4752-a2be-4988409bb675'
# df_p1 = df_survey[cols]
# df_p2 = df_p1.groupby(['SurveyDay', 'Status'])['Status'].count()
# df_p2

In [122]:
## get individual participant dataframe (for sanity check)
# df_p1.drop_duplicates()
# df_p1 = df_p1.pivot_table(index=['SurveyDay', 'ParticipantIdentifier'],
#                         columns='SurveyName', 
#                         values='Status',
#                         aggfunc=lambda x: ' '.join(x)).reset_index()